In [17]:
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from scipy.special import boxcox1p

In [28]:
## データ読み込み
train_org = pd.read_csv("../data/train_set.csv", low_memory=False)
test_org = pd.read_csv("../data/test_set.csv", low_memory=False)

In [29]:
## データの準備
df_all = pd.concat([train_org, test_org], axis=0).reset_index(drop=True)
df_all = df_all.drop(["PRICE"], axis=1)

all_r = df_all.query('SOURCE=="Residential"')
all_c = df_all.query('SOURCE=="Condominium"')

In [30]:
nc_resi = ['AYB',
'BATHRM',
'BEDRM',
'BLDG_NUM',
# 'CENSUS_TRACT',
'EYB',
'FIREPLACES',
'GBA',
'HF_BATHRM',
'KITCHENS',
'LANDAREA',
'LATITUDE',
'LONGITUDE',
'NUM_UNITS',
'ROOMS',
'STORIES',
'YR_RMDL']

nc_cond = ['AYB',
'BATHRM',
'BEDRM',
# 'CENSUS_TRACT',
'CMPLX_NUM',
'EYB',
'FIREPLACES',
'HF_BATHRM',
'LANDAREA',
'LATITUDE',
'LIVING_GBA',
'LONGITUDE',
'ROOMS',
# 'X',
# 'Y',
'YR_RMDL']

In [31]:
resi_numeric_feats = all_r[nc_resi]

skewed_feats = resi_numeric_feats.apply(lambda x: skew(x.dropna())).sort_values(ascending=False)

skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(15)

,Skew
STORIES,172.654632
BLDG_NUM,64.286192
LANDAREA,10.783696
KITCHENS,7.355380
NUM_UNITS,3.277833
GBA,2.698246
FIREPLACES,2.110796
ROOMS,1.785954
BEDRM,1.542132
EYB,1.180026


In [32]:
skewness = skewness[abs(skewness) > 0.75]

In [36]:
skewed_features = skewness.index

list(skewed_features)

['STORIES',
 'BLDG_NUM',
 'LANDAREA',
 'KITCHENS',
 'NUM_UNITS',
 'GBA',
 'FIREPLACES',
 'ROOMS',
 'BEDRM',
 'EYB',
 'BATHRM',
 'AYB',
 'HF_BATHRM',
 'LONGITUDE',
 'LATITUDE',
 'YR_RMDL']

In [24]:
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_r[feat] = boxcox1p(all_r[feat], lam)